# ML Pipeline testing
---
---

# Imports

#### Standard library imports

In [13]:
import sys
import pickle

sys.path.append("../../")

#### Third party imports

In [2]:
import pandas as pd

#### Local application imports

In [16]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
from pkg_dir.src.parameters import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extraction

In [4]:
extract_pipeline_func()

## Results

In [11]:
pkl_obj = pipeline_pkl_extract_path + "/" + pipeline_pkl_extract_name

In [14]:
with open(pkl_obj, 'rb') as obj_content:
    dfx = pickle.load(obj_content)

# Notes

---

---